# Main Process
Stochastic Model + Nonlinear Model + Resilience Assessment

In [1]:
# import modules
import numpy as np
import pandas as pd
import openseespy as ops
import pathlib
import pickle
import random
# module for SGMM

# module for NTHA
from BuildingObject import Building_object
from beam_component import Beam
from column_component import Column
from steel_material import SteelMaterial
from nonlinear_analysis import NonlinearAnalysis
from StochasticGroundMotionModeling import StochasticGroundMotionModeling
from loss_calculation import Data

In [2]:
# BASE INFORMATION
cwdFile = pathlib.Path.cwd()
buildingDataFile = cwdFile /'BuildingData'
geometryFile = buildingDataFile / 'Geometry.csv'
memberSizeFile = buildingDataFile / 'MemberSize.csv'
loadsFile = buildingDataFile / 'Loads.csv'

member_size = pd.read_csv(memberSizeFile)
gravity_loads = pd.read_csv(loadsFile)
directory = {}
directory = {'building data': buildingDataFile}
building = Building_object(directory, member_size, gravity_loads)

# beams
beamSizeFile = buildingDataFile / 'beamsectionsize.csv'
SectionDatabaseFile = cwdFile / 'AllSectionDatabase.csv'
SectionDatabase = pd.read_csv(SectionDatabaseFile)

steel = SteelMaterial(yield_stress=50, ultimate_stress=65, elastic_modulus=29000, 
                      Ry_value=1.1)  # Unit: ksi
# 创建包含梁信息的嵌套字典
beam_section_size = pd.read_csv(beamSizeFile)
beams = {}
length = int(building.geometry['X bay width'])
for _, row in beam_section_size.iterrows():
    level, bay = map(int, row[:2])
    bsection_size = {'size': row[2]}
    beams.setdefault(level, {})
    beams[level].setdefault(bay, {})
    beams[level][bay] = Beam(bsection_size['size'], length, steel,SectionDatabase)
    
# elastic demand
elasticDemandFile = cwdFile / 'elastic_demand.pkl'
with open(elasticDemandFile, 'rb') as f:
    elastic_demand = pickle.load(f)

# column
columnSizeFile = buildingDataFile / 'columnsectionsize.csv'
SectionDatabaseFile = cwdFile / 'AllSectionDatabase.csv'
SectionDatabase = pd.read_csv(SectionDatabaseFile)

steel = SteelMaterial(yield_stress=50, ultimate_stress=65, elastic_modulus=29000, 
                      Ry_value=1.1)  # Unit: ksi

# 构建包含柱信息的嵌套字典
column_section_size = pd.read_csv(columnSizeFile)
columns = {}
for _, row in column_section_size.iterrows():
    story, pier = map(int, row[:2])
    csection_size = {'size': row[2]}
    columns.setdefault(story, {})
    columns[story].setdefault(bay, {})
    axial_demand = abs(elastic_demand.dominate_load['column axial'][story, 2 * pier])
    Lx = (building.geometry['floor height'][story+1] - building.geometry['floor height'][story]).item()
    Ly = Lx
    columns[story][pier] = Column(csection_size['size'], axial_demand, Lx, Ly, steel, SectionDatabase)

baseFile = cwdFile

In [3]:
nSample = 1
edpOutput = np.zeros((nSample, 8))
costOutput = np.zeros(nSample)
np.random.seed(2)  # 确保结果可以复现
for i in range(nSample):
    print(i)
    # whichone = random.randint(0, 999)
    ACC, tn = StochasticGroundMotionModeling(7.62, 19.3, 602, 0)
    # NTHA
    dt = 0.01
    ACC = ACC.tolist()
    edpResult = NonlinearAnalysis(building, columns, beams, baseFile, ACC, dt)
    edpOutput[i, :] = edpResult
    print(edpResult)
    data = Data()
    IDR = edpResult[:3]
    PFA = edpResult[3:7]
    frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)
    print(frameCost)
    costOutput[i] = frameCost

0


Modified Ibarra-Medina-Krawinkler Model with Bilinear Hysteretic Response


Analysis Completed!
[0.00487869 0.00476931 0.00265371 0.28023853 0.19000926 0.19075288
 0.22818147 0.0010101 ]
72814.20669990484


## 验证新的loss_calculation.py 代码可以实现costOut功能就，可以对于一次给定的EDP进行千次分析，并给出最终的结果

In [1]:
import numpy as np
from loss_calculation import Data
n = 100
c = np.empty(n)
data = Data()
IDR = np.array([0.00487869, 0.00476931, 0.00265371])
PFA = np.array([0.28023853, 0.19000926, 0.19075288, 0.22818147])
RIDR = 0.0010101
costReplace = 1e6
nSample = 1000
costOut_list, costOut_mean = data.costOut(IDR, PFA, RIDR, costReplace, nSample)
print(costOut_mean)
costOut_list`

SyntaxError: invalid syntax (1981188282.py, line 13)

In [5]:
import numpy as np
from loss_calculation import Data
data = Data()
IDR_m = np.array([10, 10, 10])
PFA_m = np.array([100, 100, 100, 100])
maxRepairCost, maxsframeCost, maxnframeCost, maxcframeCost = data.cal_repair(IDR_m, PFA_m, 1000, worstCase=0)
print(np.median(maxRepairCost))
maxRepairCost

7607830.292136099


array([7412744.004069  , 6976285.09617961, 7155727.55754019,
       7772324.70374987, 7205460.11529937, 7800320.62522215,
       7670324.14590735, 8088471.97337092, 7637319.3768067 ,
       7517392.23576143, 7939371.6881365 , 7531548.32966009,
       7523739.95160722, 7250073.69928926, 7165807.84317358,
       7289021.03182572, 7631028.55781802, 7839768.77560899,
       7360268.14776642, 7603944.56282514, 7076128.87502158,
       7225890.65371723, 7740958.01499158, 6960381.48423151,
       7381254.90277819, 7478928.10363766, 7455992.40484958,
       7821375.94793933, 8087449.78394918, 7672410.30677891,
       7403935.85009692, 7913992.36948018, 7195664.06927656,
       8015553.76241633, 7847823.184846  , 7494864.17472078,
       8075810.56215286, 7489542.08977448, 7944076.42139393,
       7634578.86337635, 7026962.34375841, 7411880.1588525 ,
       7543929.25267652, 8321119.77657328, 7634790.7456124 ,
       7536328.59975675, 8133001.58290971, 7373653.78236598,
       8138941.08927587,

In [8]:
import scipy.stats as stats

# 原始的转换因子
factor_0_10 = 0.872
factor_0_90 = 1.128

# 原始的CV和均值
cv_original = 0.10
mean_original = 1.0

# 计算原始的标准差
std_original = mean_original * cv_original

# 新的CV
cv_new = 0.30

# 计算新的标准差
std_new = std_original / cv_original * cv_new

# 计算新的均值
# mean_new = mean_original * factor_0_10 / factor_0_90 * std_original

# 计算新的转换因子
cdf_0_10 = stats.norm.ppf(0.10, loc=mean_original, scale=0.3)
cdf_0_90 = stats.norm.ppf(0.90, loc=mean_original, scale=0.3)

# 打印结果
print("新的转换因子0.10：", cdf_0_10)
print("新的转换因子0.90：", cdf_0_90)


新的转换因子0.10： 0.61553453033662
新的转换因子0.90： 1.38446546966338


In [ ]:
# import pickle
# import os
# os.chdir(cwdFile)
# file = open('edpOutput.pickle', 'wb')
# pickle.dump(edpOutput, file)
# file.close()

# file = open('costOutput.pickle', 'wb')
# pickle.dump(costOutput, file)
# file.close()

In [ ]:
# with open('edpOutput.pickle', 'rb') as file:
#     dict_get = pickle.load(file)
# print(dict_get)

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(costOutput, bins=50)

In [ ]:
# IDR = np.array([4.89864183e-03, 4.80458308e-03, 2.76950154e-03])
# PFA = np.array([3.77184274e-01, 1.96116983e-01, 1.82918742e-01, 2.32016048e-01])
# data = Data()
# np.random.seed(1)  # 确保结果可以复现
# frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)
# frameCost

In [ ]:
# # SGMM
# from StochasticGroundMotionModeling import StochasticGroundMotionModeling
# import matplotlib.pyplot as plt
# # StochasticGroundMotionModeling(M, R, Vs, whichone, num=100, tn=40, F=1)
# ACC, tn = StochasticGroundMotionModeling(6.61, 19.3, 602, 0)
# t = np.arange(0, tn + 0.01, 0.01)
# plt.plot(t, ACC, linewidth=0.5)
# plt.xlabel('time (s)')
# plt.ylabel('acc (g)')
# plt.title('acc history')
# plt.grid(True)
# plt.show()

In [ ]:
# # NTHA
# dt = 0.01
# ACC = ACC.tolist()
# edpResult = NonlinearAnalysis(building, columns, beams, baseFile, ACC, dt)

In [ ]:
# print(edpResult)

In [ ]:
# edpResult[3:7]

In [ ]:
# from loss_calculation import Data
# data = Data()
# IDR = edpResult[:3]
# PFA = edpResult[3:7]
# frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)

In [ ]:
# frameCost

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# t = np.arange(0, 4001*0.01+0.002, 0.001)

# print(t.shape)
# print(U.shape)
# plt.figure(figsize = (12,4))
# plt.plot(t, U, linewidth = 1)